In [38]:
import json, csv
from csv import writer, reader

In [39]:
def save2json(filename, dump):
    out_file = open(filename, "w")
    json.dump(dump, out_file, indent = 6)
    out_file.close()

In [40]:
def save2csv(filename, header, data):
    with open(filename, 'w', newline='') as csvfile:
        csvw = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        csvw.writerow(header)
        for d in data:
            csvw.writerow(d)

In [41]:
def get_textual_entities(infile,entity_file,outfile):
    memes_entities = []
    cache = []
    errors = []
    with open(infile) as csvfile:
        urls = reader(csvfile)
        #Loading file with dbpedia entities
        with open(entity_file) as efile:
            data = json.load(efile)
            #Mapping DBPedia entities to Wikidata
            with open('data/dbpedia-wikidata.json') as jsonfile:
                db2wdmapping = json.load(jsonfile)
                for row in urls:
                    if(row[1] in data.keys() and not row[1] in cache):
                        cache.append([row[1]])
                        newdata = data[row[1]]
                        newdata['url']=row[1]
                        if("Resources" in newdata.keys()):
                            for r in newdata["Resources"]:
                                res = r['@URI'].replace('http://dbpedia.org/resource/','')
                                if(res in db2wdmapping.keys()):
                                    r['QID'] = 'https://www.wikidata.org/wiki/'+db2wdmapping[res]                       
                                if(type(r["@types"])==str):
                                    ls = r["@types"].replace("DUL:", "http://www.loa-cnr.it/ontologies/DOLCE-Lite#").replace("Wikidata:", "https://www.wikidata.org/wiki/").replace("Schema:", "https://schema.org/").split(",")
                                    ls = list(filter(None, ls))
                                    ls = [l for l in ls if 'DBpedia' not in l]
                                # elif(type(r["@types"])==list):
                                    # ls = list(map(lambda s: s.replace("DUL:", "http://www.loa-cnr.it/ontologies/DOLCE-Lite#").replace("Wikidata:", "https://www.wikidata.org/wiki/").replace("Schema:", "https://schema.org/").replace("DBpedia:", ""),r["@types"]))
                                    # errors.append([row[1],res,r["@types"]])
                                #Removing dbpedia types that do not have a correspondence in wikidata
                                for l in ls:
                                    if(l in db2wdmapping.keys()):
                                        l = 'https://www.wikidata.org/wiki/'+db2wdmapping[l]     
                                r["@typeList"]=ls
                        memes_entities.append(data[row[1]])
    save2json(outfile, memes_entities)
    save2csv('errors.csv',['kym','entity','@types'],errors)

In [42]:
def get_vision_entities(infile,entities,entity_file,outfile):
    vision = []
    cache = []
    keys = entities.keys()
    with open(infile) as csvfile:
        urls = reader(csvfile)
        with open(entity_file) as memes:
            data = json.load(memes)
            for row in urls:
                if(row[1] in data.keys() and not row[1] in cache):
                    cache.append([row[1]])
                    newdata = data[row[1]]
                    newdata['url']=row[1]
                    if('labelAnnotations' in newdata.keys()):
                        for la in newdata['labelAnnotations']:
                            if(la['mid'] in keys):
                                la['qid']=entities[la['mid']]
                    if('webDetection' in newdata.keys() and 'webEntities' in newdata['webDetection'].keys()):
                        for la in newdata['webDetection']['webEntities']:
                             if(la['entityId'] in keys):
                                la['qid']=entities[la['entityId']]
                    vision.append(data[row[1]])
    save2json(outfile, vision)

### Gets Spotlights entities (mapped to wikidata) associated to the KYM media frame seed subset

In [47]:
spotlight_enrichment_raw = 'data/raw/kym.spotlight.raw.json'

In [ ]:
raw_data  = json.load(open('data/raw/kym.spotlight.raw.json'))

In [49]:
get_textual_entities('data/seeds.to.kym.media.frames.csv',
                     spotlight_enrichment_raw,
                     'data/kym.seed.media.frames.textual.enrichment.json')

#### Tags

In [43]:
get_textual_entities('data/seeds.to.kym.media.frames.csv',
                     'data/raw/kym.spotlight.raw.tags.json',
                     'data/kym.seed.media.frames.tags.enrichment.json')

### Gets Spotlights entities (mapped to wikidata) associated to the KYM media frame 

In [48]:
get_textual_entities('data/kym.media.frames.csv',
                     spotlight_enrichment_raw,
                     'data/kym.media.frames.textual.enrichment.json')

In [44]:
get_textual_entities('data/kym.media.frames.csv',
                     'data/raw/kym.spotlight.raw.tags.json',
                     'data/kym.media.frames.tags.enrichment.json')

### Gets Google Vision entities associated to KYM subset from Wikidata

In [32]:
entities = json.load(open('data/P646.json'))

In [33]:
get_vision_entities('data/seeds.to.kym.media.frames.csv', entities,
                    'data/raw/kym.vision.raw.json', 
                    'data/kym.seed.media.frames.visual.enrichment.json')

### Gets Google Vision entities associated to KYM subset from Wikidata

In [36]:
get_vision_entities('data/kym.media.frames.csv', entities,
                    'data/raw/kym.vision.raw.json', 
                    'data/kym.media.frames.visual.enrichment.json')

In [29]:
P646 = {}
with open('data/P646.nt') as csvfile:
    triples = reader(csvfile, delimiter =' ')
    for t in triples:
        if(t[2] not in P646.keys()):
            P646[t[2]]=t[0].replace('<','').replace('>','')

In [31]:
save2json('data/p646.json', P646)